In [1]:
import xarray as xr
import pandas as pd
import os
import numpy as np

from ML_model import ML_model, SARIMA_model


In [2]:
import timeit

start_time = timeit.default_timer()


SARIMA_params = {'order': (2,0,2),             # Defining the regular AR, I and MA dependencies
                 'seasonal_order': (1,0,1,52), # Defining the seasonal dependencies
                 'trend': 'c'                  # Adding an intercept term
                 }

sarima_model = SARIMA_model(machine='local', **SARIMA_params)
sf_ds = sarima_model.run_model(data_file='vars_per_eco_update.nc')

print(sf_ds)
sf_file = sarima_model.SF_DIR + 'SARIMA_sf.nc'

res_df = sarima_model.test_model(sf_ds)
results_file = sarima_model.RESULTS_DIR + 'SARIMA_results.pkl'
print(res_df)
# res_df.to_pickle(results_file)
stop_time = timeit.default_timer()

print(f'Elapsed time: {stop_time-start_time}')


finished loading model ./models/SARIMA/2000/SARIMA_1.0.pklfinished loading model ./models/SARIMA/2000/SARIMA_7.0.pkl

finished loading model ./models/SARIMA/2000/SARIMA_8.0.pklfile "./models/SARIMA/2000/SARIMA_1.0.pkl" has already contains a trained model. Skipping training process
finished loading model ./models/SARIMA/2000/SARIMA_4.0.pklfinished loading model ./models/SARIMA/2000/SARIMA_3.0.pklfile "./models/SARIMA/2000/SARIMA_7.0.pkl" has already contains a trained model. Skipping training process
file "./models/SARIMA/2000/SARIMA_8.0.pkl" has already contains a trained model. Skipping training process


file "./models/SARIMA/2000/SARIMA_4.0.pkl" has already contains a trained model. Skipping training process

file "./models/SARIMA/2000/SARIMA_3.0.pkl" has already contains a trained model. Skipping training process
finished loading model ./models/SARIMA/2001/SARIMA_4.0.pkl
file "./models/SARIMA/2001/SARIMA_4.0.pkl" has already contains a trained model. Skipping training process
fini

In [9]:
sf_ds.where(sf_ds.testing_time.notnull() , drop=True)

<xarray.Dataset>
Dimensions:           (eco_regions: 5, time: 208, n_train_years: 17)
Coordinates:
  * time              (time) datetime64[ns] 2017-01-07 2017-01-14 ... 2020-12-26
  * n_train_years     (n_train_years) int64 1 2 3 4 5 6 7 ... 12 13 14 15 16 17
  * eco_regions       (eco_regions) float64 1.0 3.0 4.0 7.0 8.0
Data variables:
    prior_flux_per_s  (eco_regions, time) float32 1.718e+06 ... 2.189e+04
    predicted_sf      (eco_regions, n_train_years, time) float64 1.086 ... 0....
    predicted_flux    (eco_regions, n_train_years, time) float64 1.866e+06 .....
    sf_per_eco        (eco_regions, time) float32 0.246 0.8773 ... 3.674 0.63
    optimized_flux    (eco_regions, time) float32 4.227e+05 ... 1.379e+04
    training_time     (n_train_years, time) datetime64[ns] NaT NaT ... NaT NaT
    testing_time      (time) datetime64[ns] 2017-01-07 2017-01-14 ... 2020-12-26
    eco_area          (eco_regions, time) float32 2.315e+12 ... 2.924e+10
    tc_region         (eco_regions, time) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
Attributes:
    Description:  Total prior flux for each eco-region
    Units:        mol s^-1

In [10]:
for n_years, data in sf_ds.groupby('tc_region'):
    print(n_years)
    print(data)

1
<xarray.Dataset>
Dimensions:           (time: 1096, n_train_years: 17, eco_regions: 5)
Coordinates:
  * time              (time) datetime64[ns] 2000-01-01 2000-01-08 ... 2020-12-26
  * n_train_years     (n_train_years) int64 1 2 3 4 5 6 7 ... 12 13 14 15 16 17
  * eco_regions       (eco_regions) float64 1.0 3.0 4.0 7.0 8.0
Data variables:
    prior_flux_per_s  (eco_regions, time) float32 1.726e+06 ... 2.189e+04
    predicted_sf      (eco_regions, n_train_years, time) float64 nan ... 0.6406
    predicted_flux    (eco_regions, n_train_years, time) float64 nan ... 1.40...
    sf_per_eco        (eco_regions, time) float32 0.2209 0.7194 ... 3.674 0.63
    optimized_flux    (eco_regions, time) float32 3.813e+05 ... 1.379e+04
    training_time     (n_train_years, time) datetime64[ns] NaT NaT ... NaT NaT
    testing_time      (time) datetime64[ns] NaT NaT ... 2020-12-19 2020-12-26
    eco_area          (eco_regions) float32 2.315e+12 5.923e+11 ... 2.924e+10
    tc_region         (eco_regions

In [2]:
# Depending on the machine on which the code is run, data might be stored in different directories.
# Indicate which machine is used to make sure the path to the data can be found.
# Can either be "local" or "Snellius"

MACHINE = 'local'

if MACHINE == 'Snellius':
    pred_var_path = '/gpfs/work1/0/ctdas/awoude/Ritten/predictor_vars/' # For retrieving the set of aggregated scaling vectors

    SAVE_DIR = pers_file_dir = '/gpfs/work1/0/ctdas/awoude/Ritten/trained_models/' # used for storing the trained model

    SF_DIR = '/gpfs/work1/0/ctdas/awoude/Ritten/fitted_sf/' # used for storing the scaling factor produced bij ML models

    RESULTS_DIR = '/gpfs/work1/0/ctdas/awoude/Ritten/results/' # used for storing the trained model

elif MACHINE == 'local':
    pred_var_path = './' # For retrieving the set of aggregated scaling vectors

    SAVE_DIR = pers_file_dir = './models/' # used for storing the trained model

    SF_DIR = './fitted_sf/' # used for storing the scaling factor produced bij ML models

    RESULTS_DIR = './results/'


transCom_dict = {1:'North American Boreal',
                   2:'North American Temperate',
                   7:'Eurasia Boreal',
                   8:'Eurasia Temperate',
                   11:'Europe',
                  }

In [3]:

# Loading all necessary data
with xr.open_dataset(pred_var_path + 'vars_per_eco_update.nc') as ds:
    complete_ds = ds


In [8]:
complete_ds.loc

Data variables:
    regions           (lat, lon) float32 240.0 240.0 240.0 ... 210.0 210.0 210.0
    grid_cell_weight  (lat, lon) float64 1.079e+08 1.079e+08 ... 1.079e+08
    grid_cell_area    (lat, lon) float32 1.079e+08 1.079e+08 ... 1.079e+08
    TER               (lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    monthly_flux      (eco_regions, time) float64 1.726e+06 ... 1.367e+05
    smoothed_flux     (eco_regions, time) float64 1.726e+06 ... 9.337e+04
    opt_flux          (eco_regions, time) float32 3.813e+05 ... 1.211e+05
    prior_flux_per_s  (eco_regions, time) float32 1.726e+06 ... 1.32e+05
    scaling_factor    (eco_regions, time) float32 146.5 231.4 ... 220.6 186.4
    sf_per_eco        (eco_regions, time) float32 0.2209 0.7194 ... 0.9176
    monthly_sf        (eco_regions, time) float64 1.0 0.2146 ... 1.036 1.035
    smoothed_sf       (eco_regions, time) float64 1.0 0.7382 ... 0.6828 0.7072
    g10m_MAX          (eco_regions, time) float64 12.99 13.69 ... 14